# PyTorch Deep Dive: Practical Example - Regression

We have learned the theory. Now let's solve a real problem.

In this notebook, we will predict a **continuous value** (e.g., House Price). This is called **Regression**.

## Learning Objectives
- **The Vocabulary**: What is "Regression", "Overfitting", and "Underfitting"?
- **The Intuition**: The "Goldilocks" analogy for model complexity.
- **The Practice**: Building a model to predict non-linear data.
- **The Visual**: Seeing the model fit the curve.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

torch.manual_seed(42)

## Part 1: The Vocabulary (Definitions First)

Before we code, let's define the problem type and the pitfalls.

### 1. Regression vs Classification
- **Regression**: Predicting a quantity (How much?).
    - Example: Price, Temperature, Height.
    - Output: A single number (e.g., 345.2).
- **Classification**: Predicting a category (Which one?).
    - Example: Cat vs Dog, Spam vs Not Spam.
    - Output: A probability (e.g., 80% Cat).

### 2. Overfitting (Memorization)
- When the model learns the *noise* instead of the *pattern*.
- It does great on training data but fails on new data.
- Analogy: Memorizing the answers to the practice test but failing the real exam.

### 3. Underfitting (Oversimplification)
- When the model is too simple to capture the pattern.
- It does poorly on everything.
- Analogy: Trying to explain Quantum Physics using only addition.

## Part 2: The Intuition (Goldilocks Principle)

Building a model is like fitting a bed for Goldilocks.

- **Too Hard (Underfitting)**: A straight line trying to fit a curve. It misses the point.
- **Too Soft (Overfitting)**: A squiggly line that touches every single dot. It's too sensitive to noise.
- **Just Right (Generalization)**: A smooth curve that captures the trend.

Our goal is to find the "Just Right" model.

## Part 3: The Data (Non-Linear)

Let's create some data that isn't a straight line. Let's use a sine wave with some noise.

In [ ]:
# Create data: y = sin(x)
x = torch.linspace(-5, 5, 100).view(-1, 1)
y = torch.sin(x) + 0.1 * torch.randn(x.size())

plt.scatter(x.numpy(), y.numpy())
plt.title("Noisy Sine Wave")
plt.show()

## Part 4: The Model (Going Deeper)

A single Linear Layer cannot learn a sine wave. It can only learn a straight line.
To learn curves, we need **Hidden Layers** and **Activation Functions**.

In [ ]:
class SineNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Layer 1: Expand to 50 neurons (Add complexity)
        self.hidden1 = nn.Linear(1, 50)
        # Layer 2: Another 50 neurons (More complexity)
        self.hidden2 = nn.Linear(50, 50)
        # Output Layer: Back to 1 number
        self.output = nn.Linear(50, 1)
        # Activation: ReLU (The bendy part)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

model = SineNet()
optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam is often better than SGD
criterion = nn.MSELoss()

## Part 5: Training (The Loop)

We use the same 5-step loop as before.

In [ ]:
epochs = 1000
for epoch in range(epochs):
    # 1. Forward
    pred = model(x)
    # 2. Loss
    loss = criterion(pred, y)
    # 3. Zero
    optimizer.zero_grad()
    # 4. Backward
    loss.backward()
    # 5. Step
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss {loss.item():.4f}")

## Part 6: Visualization (The Moment of Truth)

Did our model learn the curve?

In [ ]:
plt.scatter(x.numpy(), y.numpy(), label="Data")
with torch.no_grad():
    plt.plot(x.numpy(), model(x).numpy(), color='red', label="Prediction", linewidth=3)
plt.title("Regression: Fitting a Curve")
plt.legend()
plt.show()

## Summary Checklist

1. **Regression** = Predicting a continuous number.
2. **Hidden Layers** = Allow the model to learn complex, non-linear patterns.
3. **Overfitting** = Memorizing noise (Bad).
4. **Underfitting** = Failing to capture the pattern (Bad).

Next, we will tackle the other main type of problem: **Classification**.